In [1]:
from langchain.text_splitter import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf


# Extract elements from PDF
def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    return partition_pdf(fname,
                       extract_images_in_pdf=True,
                       infer_table_structure=True,
                       strategy="auto",
                        max_characters=4000,
                        new_after_n_chars=3800,
                        combine_text_under_n_chars=2000
    )


# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables


# File path
fpath = "/project/code/Attention_is_all_you_need.pdf"
fname = "Attention_is_all_you_need.pdf"

# Get elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

/home/workbench/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the che

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [3]:
import base64
import os

from langchain_core.messages import HumanMessage


def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4o", max_tokens=200)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    from tqdm import tqdm
    import time

    prompt = """You are an assistant tasked with summarizing images for retrieval. \
        These summaries will be embedded and used to retrieve the raw image. \
        Give a concise summary of the image that is well optimized for retrieval."""

    count = 0

    for img_file in tqdm(sorted(os.listdir(path)), desc="Processing images"):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)

            try:
                base64_image = encode_image(img_path)
                img_base64_list.append(base64_image)
                image_summaries.append(image_summarize(base64_image, prompt))
                count += 1

            except Exception as e:
                print(f"Error processing image {img_file}: {e}")

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries("/project/data")

Processing images: 100%|██████████| 2/2 [00:00<00:00, 45590.26it/s]


In [4]:
import uuid, os

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


# The vectorstore to use to index the summaries
vectorstore = Chroma(
    collection_name="rag-storage", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

/tmp/ipykernel_2318/2114994243.py:53: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


In [5]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are a deep learning and machine learning specialist.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts.\n"
            "Use this information to provide quality information related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


# Create RAG chain
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [6]:
# Check retrieval
query = "Can you give me a brief description on the document."
docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)

# We get 4 docs
len(docs)

/tmp/ipykernel_2318/2728129363.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)


4

In [7]:
# Run RAG chain
response = chain_multimodal_rag.invoke(query)
response = response.split('.')

# Print each line in a new line
for line in response:
    print(line)

The document appears to be a detailed comparison and analysis of various machine learning models, particularly focusing on neural network architectures and their performance metrics
 Here's a brief description of the key sections and their contents:

1
 **Model Configuration and Performance Metrics**:
   - The table lists different configurations of a base model with parameters such as the number of layers (N), model dimension (dmodel), feed-forward dimension (dff), number of attention heads (h), key dimension (dk), value dimension (dv), dropout rate (Pdrop), label smoothing (ϵls), training steps, perplexity (PPL) on the development set, BLEU score, and the number of parameters (params)

   - Different configurations (A, B, C, D, E) are compared based on these metrics, showing how changes in model architecture and hyperparameters affect performance


2
 **BLEU Scores and Training Costs**:
   - This section compares the BLEU scores and training costs (in FLOPs) of various models for Eng

In [8]:
response = chain_multimodal_rag.invoke("What layer comes after the Add & Norm of the Multi-head Attention mechanism of the transformer model architecture?")
response = response.split(".")
for line in response:
    print (line)

In the transformer model architecture, the layer that comes after the Add & Norm layer following the Multi-head Attention layer is the Feed-Forward Neural Network (FFN) layer
 

Here is a brief overview of the sequence of layers in a transformer model:

1
 **Multi-head Self-Attention**: This layer allows the model to focus on different parts of the input sequence simultaneously

2
 **Add & Norm**: The output of the Multi-head Self-Attention layer is added to the original input (residual connection) and then normalized

3
 **Feed-Forward Neural Network (FFN)**: This layer consists of two linear transformations with a ReLU activation in between
 It is applied to each position separately and identically


The sequence can be summarized as follows:
- Multi-head Self-Attention
- Add & Norm
- Feed-Forward Neural Network (FFN)
- Add & Norm

This pattern is repeated in each layer of the transformer model



In [9]:
response = chain_multimodal_rag.invoke("What layer comes after the optional Mask layer in the scaled dot-product attention block?")
response = response.split(".")
for line in response:
    print (line)

In the context of the scaled dot-product attention mechanism, the layer that typically follows the optional Mask layer is the **Softmax layer**
 

Here's a brief overview of the process:

1
 **Query, Key, and Value Matrices**: The input is transformed into three matrices: Query (Q), Key (K), and Value (V)

2
 **Scaled Dot-Product**: The dot product of the Query and Key matrices is computed and scaled by the square root of the dimension of the Key matrix

3
 **Optional Masking**: A mask can be applied to the scaled dot-product to prevent attending to certain positions (e
g
, future positions in the case of decoder self-attention in transformers)

4
 **Softmax**: The result of the scaled dot-product (after optional masking) is passed through a Softmax layer to obtain the attention weights

5
 **Weighted Sum**: The attention weights are then used to compute a weighted sum of the Value matrix


So, the sequence of operations in the scaled dot-product attention block is:

1
 Compute \( QK^T

In [10]:
response = chain_multimodal_rag.invoke("What is the BLEU score for ByteNet model?")
response = response.split(".")
for line in response:
    print (line)

The BLEU score for the ByteNet model, as provided in the text, is 23
75 for the EN-DE (English to German) translation task
 The table does not provide a BLEU score for the EN-FR (English to French) translation task for the ByteNet model
 

Here is the relevant excerpt from the table for clarity:

```
Model          BLEU EN-DE  EN-FR
ByteNet [18]   23
75
```

This indicates that the ByteNet model achieved a BLEU score of 23
75 on the English to German translation task



In [11]:
response = chain_multimodal_rag.invoke("What is the BLEU score for ConvS2S model?")
response = response.split(".")
for line in response:
    print (line)

The BLEU score for the ConvS2S (Convolutional Sequence to Sequence) model varies depending on the language pair and whether an ensemble model is used
 Based on the provided data:

- For the English-to-German (EN-DE) translation task:
  - The ConvS2S model achieves a BLEU score of **25
16**

  - The ConvS2S Ensemble model achieves a BLEU score of **26
36**


- For the English-to-French (EN-FR) translation task:
  - The ConvS2S model achieves a BLEU score of **40
46**

  - The ConvS2S Ensemble model achieves a BLEU score of **41
29**


These scores indicate the performance of the ConvS2S model in terms of translation quality, with the ensemble models generally performing better than the single models



In [12]:
response = chain_multimodal_rag.invoke("What is the complexity of the Maximum Path Lengths for a convolutional layer type?")
response = response.split(".")
for line in response:
    print (line)

The complexity of the Maximum Path Lengths for a convolutional layer type can be derived from the provided table
 Let's break down the information:

### Complexity of Maximum Path Lengths for Different Layer Types

| Layer Type                | Complexity per Layer | Sequential Maximum Path Length | Operations |
|---------------------------|----------------------|--------------------------------|------------|
| Self-Attention            | O(n² · d)            | O(1)                           | O(1)       |
| Recurrent                 | O(n · d²)            | O(n)                           | O(n)       |
| Convolutional             | O(k · n · d²)        | O(1)                           | O(logₖ(n)) |
| Self-Attention (restricted)| O(r · n · d)         | O(1)                           | O(n/r)     |

### Explanation

- **Layer Type**: This column lists the type of neural network layer

- **Complexity per Layer**: This column shows the computational complexity for each layer type

- **Se